# Pre-Processing

In this notebook, we load in the complete dataset ([2020 Global State of Democracy Index](https://www.idea.int/gsod-indices/dataset-resources), which has data from 1975-2019) and split it out into train, test, and query sets for both the data as a whole and the most current year for the purposes of machine learning training.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/gsodi_pv_4.csv', dtype={
    'ID': int, 'ID_country_name': 'category', 'ID_country_code': int, 'ID_year': int, 
    'ID_country_year': int, 'ID_region': 'category', 'ID_subregion': 'category',
})

In [3]:
# data.head()
# data.describe()

## Current Year

Here we will split out the most current year of data (2019) into separate files and training sets. We go ahead and load the complete set of 2019 data into

In [4]:
data_2019 = data[data['ID_year'] == 2019]
data_2019.to_csv('data/complete-2019.csv')

Though we are typically wanting to do analysis at the country level, several rows in our dataset aggregate the information up to a World, region, or sub-region level. Here we drop all of the regions and save a set of the data which only includes the countries.

In [5]:
drop_regions = ['African Union', 'ASEAN', 'European Union', 'OECD', 'OAS', 'East Africa', 'Central Africa', 
                'Southern Africa', 'West Africa', 'North Africa', 'Caribbean', 'Central America', 'South America', 
                'North America', 'Central Asia', 'East Asia','South Asia', 'South-East Asia', 'Oceania', 
                'Middle East', 'East-Central Europe', 'Eastern Europe', 'North/Western Europe', 'Southern Europe', 
                'Africa', 'Latin America/Caribbean','North America', 'Asia/Pacific', 'Middle East', 'Europe', 'World']
countries_2019 = data_2019[-data_2019['ID_country_name'].isin(drop_regions)]
countries_2019.to_csv('data/countries-2019.csv')
# countries_2019

Next, we go ahead and split out into test, query, and training datasets. We want to have a 60-20-20 split for train-query-test sets, and additionally an 80-20 train-test split for any models we create without requiring a query set. To do this, we first create `test_2019` with our 20% test set, and the complementary `train80_2019` which has the other 80% of the data for training in an 80-20 split. The `random_state` seed may be dropped from the `sample` call for an added layer of randomness, but for the sake of reproducibility we keep the seed here.

In [6]:
test_2019 = countries_2019.sample(frac=0.2, random_state=0)
train80_2019 = countries_2019.drop(test_2019.index)

To create the rest of what we need for a 60-20-20 split, we split `train80_2019` into `train_2019` and `query_2019` which will be 60% and 20% of the complete 2019 countries dataset, respectively.

In [7]:
train_2019 = train80_2019.sample(frac=0.75, random_state=0)
query_2019 = train80_2019.drop(train_2019.index)

Finally, we can save these datasets into csv format for future use.

In [8]:
train_2019.to_csv('data/train-2019.csv')
query_2019.to_csv('data/query-2019.csv')
train80_2019.to_csv('data/train80-2019.csv')
test_2019.to_csv('data/test-2019.csv')

## Countries Over Time

We now repeat a similar process as above, but without reducing the dataset to a single year. We drop the rows that describe regions from our dataset, and create our 80-20 and 60-20-20 splits as above.

In [9]:
complete_countries = data[-data['ID_country_name'].isin(drop_regions)]
test_countries = complete_countries.sample(frac=0.2, random_state=0)
train80_countries = complete_countries.drop(test_countries.index)

In [10]:
train_countries = train80_countries.sample(frac=0.75, random_state=0)
query_countries = train80_countries.drop(train_countries.index)

In [11]:
complete_countries.to_csv('data/complete-countries.csv')
train_countries.to_csv('data/train-countries.csv')
query_countries.to_csv('data/query-countries.csv')
train80_countries.to_csv('data/train80-countries.csv')
test_countries.to_csv('data/test-countries.csv')

In [12]:
complete_countries

,ID,ID_country_name,ID_country_code,ID_year,ID_country_year,ID_region,ID_subregion,C_A1,L_A1,U_A1,...,v_51_02,v_51_03,v_51_04,v_51_05,v_51_06,v_52_01,v_53_01,v_53_02,v_54_01,v_54_02
0,1,United States,2,1975,21975,North America,North America,0.753310,0.687280,0.819340,...,0.827091,0.773302,NaN,NaN,NaN,0.3823,0.000000,1.0,0.957789,0.819947
1,2,United States,2,1976,21976,North America,North America,0.769298,0.705428,0.833167,...,0.827091,0.885387,NaN,NaN,NaN,0.5692,0.000000,1.0,0.957789,0.819947
2,3,United States,2,1977,21977,North America,North America,0.838041,0.772376,0.903706,...,0.837689,0.885387,NaN,NaN,NaN,0.5692,0.000000,1.0,0.957789,0.819947
3,4,United States,2,1978,21978,North America,North America,0.836824,0.771909,0.901739,...,0.837689,0.885387,NaN,NaN,NaN,0.3720,0.000000,1.0,0.957789,0.819947
4,5,United States,2,1979,21979,North America,North America,0.834315,0.768308,0.900322,...,0.837689,0.885387,NaN,NaN,NaN,0.3720,0.000000,1.0,0.957789,0.819947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6852,6853,New Zealand,920,2015,9202015,Asia/Pacific,Oceania,0.884517,0.819679,0.949355,...,0.712398,0.715490,0.524851,0.569213,0.46073,0.7235,0.519846,1.0,0.977889,0.901362
6853,6854,New Zealand,920,2016,9202016,Asia/Pacific,Oceania,0.885860,0.818061,0.953660,...,0.712398,0.750716,0.524851,0.569213,0.46073,0.7235,0.519846,1.0,0.977889,0.901362
6854,6855,New Zealand,920,2017,9202017,Asia/Pacific,Oceania,0.888417,0.821074,0.955759,...,0.687427,0.746166,0.524851,0.569213,0.46073,0.7635,0.519846,1.0,0.969849,0.876624
6855,6856,New Zealand,920,2018,9202018,Asia/Pacific,Oceania,0.847438,0.781873,0.913002,...,0.714866,0.748188,0.524851,0.569213,0.46073,0.7635,0.491677,1.0,0.969849,0.884140


## Population

For some visualizations, we also want to pull in some population data. For this, we pull in some [data from The World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL), but it needs cleaned up before it is useful to us. Here, we remove some header lines, tidy the data, and filter down to only the years we care about.

In [28]:
years_to_drop = [ str(i) for i in range(1960, 1975) ]

In [30]:
population = pd.read_csv('data/API_SP.POP.TOTL_DS2_en_csv_v2_1865227.csv', 
                         skiprows = 4).drop(
    columns = ['Country Code', 'Indicator Name', 'Indicator Code', 
               '2020', 'Unnamed: 65']).drop(columns = years_to_drop)
population

,Country Name,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,60567.0,61345.0,62201.0,...,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0
1,Afghanistan,12689160.0,12943093.0,13171306.0,13341198.0,13411056.0,13356511.0,13171673.0,12882528.0,12537730.0,...,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0
2,Angola,7024000.0,7279509.0,7533735.0,7790707.0,8058067.0,8341289.0,8640446.0,8952950.0,9278096.0,...,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0
3,Albania,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,...,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0
4,Andorra,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,37500.0,39114.0,40867.0,...,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1369000.0,1400000.0,1430000.0,1460000.0,1491000.0,1521000.0,1552000.0,1582000.0,1614000.0,...,1775680.0,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1794248.0
260,"Yemen, Rep.",6784695.0,6967941.0,7178675.0,7414158.0,7669694.0,7941898.0,8231910.0,8541605.0,8869370.0,...,23154855.0,23807588.0,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0,28498687.0,29161922.0
261,South Africa,25195187.0,25836888.0,26480913.0,27138965.0,27827320.0,28556769.0,29333103.0,30150448.0,30993758.0,...,51216964.0,52004172.0,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0,57779622.0,58558270.0
262,Zambia,4943283.0,5112823.0,5287548.0,5468262.0,5656139.0,5851825.0,6055366.0,6265864.0,6481916.0,...,13605984.0,14023193.0,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0,17351822.0,17861030.0


In [35]:
population_long = pd.melt(population, id_vars = ['Country Name'])
population_long = population_long.rename(columns = {'Country Name': 'Country',
                                                    'variable': 'Year',
                                                    'value': 'population'})
population_long.to_csv('data/population.csv')